<a href="https://colab.research.google.com/github/Jonathan-C-Barrett/GEOG5990M/blob/main/GEOG5003M_Final_Project_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GEOG5003M Final Assignment (Template)

Student ID number:

GitHub repo link:

Word count limit= 1,500 words max (markdown cells only, excluding readme)

In [66]:
# read in required packages

import geopandas as gpd
import pandas as pd
import seaborn as sns
from scipy import stats
import numpy as np

import matplotlib.pyplot as plt

!pip install mapclassify

In [67]:
#Read in Health Services Data
ahah = pd.read_csv('https://github.com/Jonathan-C-Barrett/GEOG5990M/raw/refs/heads/main/ahah_v4.csv')
ahah.head()

,LSOA21CD,ah4blue,ah4dent,ah4gp,ah4hosp,ah4phar,ah4leis,ah4pubs,ah4ffood,ah4gpas,...,ah4g_rnk,ah4e_rnk,ah4r_rnk,ah4h_pct,ah4g_pct,ah4e_pct,ah4r_pct,ah4ahah,ah4ahah_rnk,ah4ahah_pct
0,E01000001,1.8048,0.6328,1.0316,0.9581,0.7487,0.3596,0.4210,0.3920,0.1056,...,40830,42589,41495,2,96,100,98,59.7543,42592,100
1,E01000002,2.1041,0.6230,1.3232,1.1370,0.6849,0.5571,0.4852,0.3946,0.0778,...,42201,42590,41673,4,99,100,98,65.4344,42635,100
2,E01000003,2.2753,0.5709,0.8562,0.7862,0.5919,0.4608,0.5679,0.5919,0.1339,...,40631,42590,40857,1,96,100,96,57.4313,42559,100
3,E01000005,1.1553,0.8337,1.1078,0.8697,0.6657,0.3717,0.2673,0.2934,0.0774,...,40765,42616,42419,2,96,100,100,64.3384,42629,100
4,E01000006,1.9055,1.9361,0.9587,1.5191,1.3641,1.5219,1.5017,1.1499,0.3310,...,23517,41369,34909,22,56,98,82,33.5326,38389,91


In [68]:
#Select only columns relating to Health Services
health = ahah[['LSOA21CD', 'ah4dent', 'ah4gp', 'ah4hosp', 'ah4phar', 'ah4leis', 'ah4h']]
health.head()

,LSOA21CD,ah4dent,ah4gp,ah4hosp,ah4phar,ah4leis,ah4h
0,E01000001,0.6328,1.0316,0.9581,0.7487,0.3596,-1.7834
1,E01000002,0.6230,1.3232,1.1370,0.6849,0.5571,-1.5523
2,E01000003,0.5709,0.8562,0.7862,0.5919,0.4608,-1.9813
3,E01000005,0.8337,1.1078,0.8697,0.6657,0.3717,-1.7260
4,E01000006,1.9361,0.9587,1.5191,1.3641,1.5219,-0.6686


In [69]:
# Data downloaded from https://geoportal.statistics.gov.uk/maps/761ecd09b4124843b95511a242e2b1a1
shp =gpd.read_file('/content/Output_Areas_2021_EW_BFE_V9_-6709771492190738846.geojson')


In [70]:
shp.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 35507 entries, 0 to 35506
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   FID        35507 non-null  int32   
 1   OA21CD     35507 non-null  object  
 2   LSOA21CD   35507 non-null  object  
 3   LSOA21NM   35507 non-null  object  
 4   LSOA21NMW  35507 non-null  object  
 5   BNG_E      35507 non-null  int32   
 6   BNG_N      35507 non-null  int32   
 7   LAT        35507 non-null  float64 
 8   LONG       35507 non-null  float64 
 9   GlobalID   35507 non-null  object  
 10  geometry   35507 non-null  geometry
dtypes: float64(2), geometry(1), int32(3), object(5)
memory usage: 2.6+ MB


In [71]:
#Select only LSOAs in London Boroughs
london_shp= shp.loc[shp['LSOA21NM'].str.contains('City of London') |
                    shp['LSOA21NM'].str.contains('Barking and Dagenham')|
                    shp['LSOA21NM'].str.contains('Barnet')|
                    shp['LSOA21NM'].str.contains('Bexley')|
                    shp['LSOA21NM'].str.contains('Brent') |
                    shp['LSOA21NM'].str.contains('Bromley')|
                    shp['LSOA21NM'].str.contains('Camden')|
                    shp['LSOA21NM'].str.contains('Croydon')|
                    shp['LSOA21NM'].str.contains('Ealing')|
                    shp['LSOA21NM'].str.contains('Enfield')|
                    shp['LSOA21NM'].str.contains('Greenwich')|
                    shp['LSOA21NM'].str.contains('Hackney')|
                    shp['LSOA21NM'].str.contains('Hammersmith and Fulham')|
                    shp['LSOA21NM'].str.contains('Haringey')|
                    shp['LSOA21NM'].str.contains('Harrow')|
                    shp['LSOA21NM'].str.contains('Havering')|
                    shp['LSOA21NM'].str.contains('Hillingdon')|
                    shp['LSOA21NM'].str.contains('Hounslow')|
                    shp['LSOA21NM'].str.contains('Islington')|
                    shp['LSOA21NM'].str.contains('Kensington and Chelsea')|
                    shp['LSOA21NM'].str.contains('Kingston upon Thames')|
                    shp['LSOA21NM'].str.contains('Lambeth')|
                    shp['LSOA21NM'].str.contains('Lewisham')|
                    shp['LSOA21NM'].str.contains('Merton')|
                    shp['LSOA21NM'].str.contains('Newham')|
                    shp['LSOA21NM'].str.contains('Redbridge')|
                    shp['LSOA21NM'].str.contains('Richmond upon Thames')|
                    shp['LSOA21NM'].str.contains('Southwark')|
                    shp['LSOA21NM'].str.contains('Sutton')|
                    shp['LSOA21NM'].str.contains('Southwark')|
                    shp['LSOA21NM'].str.contains('Tower Hamlets')|
                    shp['LSOA21NM'].str.contains('Waltham Forest')|
                    shp['LSOA21NM'].str.contains('Wandsworth')|
                    shp['LSOA21NM'].str.contains('Westminster'),:]

TypeError: StringMethods.contains() takes from 2 to 6 positional arguments but 34 were given

This is an example block of mardown text I want to reference <a href="#ref1">[1]</a>. I might need to add some more citations <a href="#ref2">[2]</a><a href="#ref2">[3]</a>



Data Visualisation
- Don't forget to present your final two data visualisation (one spatial and one non-spatial) and the justifications about the decisions you made whilst preparing and visualising the data.

## References
<p><a href="add_url_here">[1]</a>Footnote citation goes here</p>
<p><a href="https://github.com/FrancescaPontin/GEOG5990">[2]</a>GEOG5990M course materials</p>
<p><a href="add_url_here">[3]</a>Citation 3</p>